# Downloading Libraries


In [26]:
! pip install nltk
! pip install vaderSentiment
! pip install pytrends
! pip install textblob
! pip install wordcloud
! pip install gensim
! pip install seaborn
! pip install TextBlob
! pip install joblib

# Importing Libraries


In [27]:
import nltk
import logging
import numpy as np
import pandas as pd
import seaborn as sns
from textblob import TextBlob
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

import re
import string
import joblib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [28]:
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('wordnet', quiet=True)

[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Importing Dataset


In [29]:
df = pd.read_csv('Dataset/Uncleaned Data/clean_reviews.csv')
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


# Data Preprocessing


In [30]:
df.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [31]:
df.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [32]:
df.shape

(1581033, 6)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581033 entries, 0 to 1581032
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1581033 non-null  int64 
 1   id             1581033 non-null  int64 
 2   date           1581033 non-null  object
 3   reviewer_id    1581033 non-null  int64 
 4   reviewer_name  1581033 non-null  object
 5   comments       1580874 non-null  object
dtypes: int64(3), object(3)
memory usage: 72.4+ MB


In [34]:
df.describe()

,listing_id,id,reviewer_id
count,1.581033e+06,1.581033e+06,1.581033e+06
mean,1.222594e+17,3.886080e+17,1.492887e+08
std,2.756938e+17,3.944920e+17,1.464763e+08
min,1.391300e+04,2.103200e+04,4.400000e+01
25%,1.114422e+07,3.152755e+08,3.168621e+07
50%,2.337278e+07,4.316597e+17,9.604527e+07
75%,4.608831e+07,7.802538e+17,2.255192e+08
max,9.723611e+17,9.745963e+17,5.355267e+08


In [35]:
df.isnull().sum()

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      0
comments         159
dtype: int64

In [36]:
df.duplicated().sum()

0

In [37]:
# remove the following columns id, date and reviewer_id
df = df.drop(['id', 'reviewer_id', 'listing_id'], axis=1)
df.head()

,date,reviewer_name,comments
0,2010-08-18,Michael,My girlfriend and I hadn't known Alina before ...
1,2011-07-11,Mathias,Alina was a really good host. The flat is clea...
2,2011-09-13,Kristin,Alina is an amazing host. She made me feel rig...
3,2011-10-03,Camilla,"Alina's place is so nice, the room is big and ..."
4,2011-10-09,Jorik,"Nice location in Islington area, good for shor..."


In [38]:
def remove_urls(text):
    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing URLs from the text
    url_pattern = r'http\S+|www\S+|https\S+'
    cleaned_text = re.sub(url_pattern, '', text, flags=re.MULTILINE)

    return cleaned_text

In [39]:
def remove_mentions_hashtags(text):
    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing mentions and hashtags from the text
    pattern = r'\@\w+|\#\w+'
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [40]:
def remove_punctuations(text):

    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing punctuations from the text
    translation_table = str.maketrans('', '', string.punctuation)
    cleaned_text = text.translate(translation_table)

    return cleaned_text

In [41]:
# Preload stop words to avoid reloading them every time the function is called
stop_words = set(stopwords.words('english'))


def remove_stopwords(text):
    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing stop words from the text
    filtered_words = [word for word in text.split() if word not in stop_words]
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [42]:
def remove_numbers(text):
    """Remove numbers from a text string"""
    return re.sub(r'\d+', '', text)

In [43]:
def remove_emojis(text):
    """Remove emojis from a text string"""
    emoji_pattern = re.compile(pattern="["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [44]:
def handle_whitespace(text):
    """Handle excessive whitespace"""
    return re.sub(r'\s+', ' ', text).strip()

In [45]:
def lemmitization(text):
    """Lemmitize text"""
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [46]:
def stemminization(text):
    """Stem text"""
    stemmer = nltk.stem.PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [47]:
def clean_text(text):
    text = text.lower()
    text = remove_urls(text)
    text = remove_mentions_hashtags(text)
    text = remove_punctuations(text)
    text = remove_stopwords(text)
    text = remove_numbers(text)
    text = remove_emojis(text)
    text = handle_whitespace(text)
    text = lemmitization(text)
    text = stemminization(text)

    # if new functions are added, add them here

    return text

In [48]:
# convert comments column to str type
df['comments'] = df['comments'].astype(str)

In [49]:
df['cleaned_comments'] = df['comments'].apply(clean_text)
df.head()

,date,reviewer_name,comments,cleaned_comments
0,2010-08-18,Michael,My girlfriend and I hadn't known Alina before ...,girlfriend hadnt known alina took leap faith r...
1,2011-07-11,Mathias,Alina was a really good host. The flat is clea...,alina realli good host flat clean tidi realli ...
2,2011-09-13,Kristin,Alina is an amazing host. She made me feel rig...,alina amaz host made feel right home like hang...
3,2011-10-03,Camilla,"Alina's place is so nice, the room is big and ...",alina place nice room big clean bed huge alina...
4,2011-10-09,Jorik,"Nice location in Islington area, good for shor...",nice locat islington area good short busi trip...


In [50]:
df.duplicated().sum()

158

In [51]:
# remove duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [52]:
# Sentiment analysis
df['polarity'] = df['cleaned_comments'].apply(
    lambda x: TextBlob(x).sentiment.polarity)
df['sentiment'] = pd.cut(df['polarity'], bins=3, labels=[
                         'negative', 'neutral', 'positive'])

In [ ]:
# print the number of positive, negative and neutral reviews
df['sentiment'].value_counts()

sentiment
positive    896748
neutral     680210
negative      3917
Name: count, dtype: int64

In [ ]:
# take 3915 positive reviews and 3915 negative reviews and 3915 neutral reviews
df = df.groupby('sentiment').apply(lambda x: x.sample(n=3915)).reset_index(
    drop=True).sort_values(by=['sentiment'])
df['sentiment'].value_counts()

sentiment
negative    3915
neutral     3915
positive    3915
Name: count, dtype: int64

In [ ]:
df = pd.read_csv('Dataset/Cleaned Data/reviews.csv')
df.head()

,date,reviewer_name,comments,cleaned_comments,polarity,sentiment
0,2016-08-20,Jenny,"Die Unterkunft war wirklicht toll, ruhig, nett...",die unterkunft war wirklicht toll ruhig nett z...,-0.7,negative
1,2023-01-08,Elia,"El apartamento estaba correcto, tal cual, como...",el apartamento estaba correcto tal cual como e...,-0.6,negative
2,2023-05-02,Cecilia,Yo buscaba estar cerca de la estación de Liver...,yo buscaba estar cerca de la estación de liver...,-0.6,negative
3,2018-11-25,Sibylle,"Elisas Wohnung liegt in Hampsted, mehrere Busl...",elisa wohnung liegt hampst mehrer buslinien ha...,-0.7,negative
4,2018-02-18,Nathalie,Ich habe meinen Aufenthalt bei Hedge und Britt...,ich habe meinen aufenthalt bei hedg und britt ...,-0.7,negative


In [ ]:
# remove any rows with null values
df = df.dropna()
df.isnull().sum()

date                0
reviewer_name       0
comments            0
cleaned_comments    0
polarity            0
sentiment           0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
# drop any rows which do not start with a number in the date column
df = df[df['date'].str.contains('^\d', na=False)]

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# store the cleaned data
df.to_csv('Dataset/Cleaned Data/reviews.csv', index=False)